In [18]:
import pandas as pd
import dotenv
dotenv.load_dotenv()

True

# Reading Data
Read in news data, drop duplicates then parse published date into Timestamp object

In [19]:
news = pd.read_csv(".data/organized_news.csv")
news.drop([news.columns[0]], axis=1, inplace=True)
news.head()

,title,description,published date,url,publisher,fetched_title,fetched_body
0,New York Times (NYT) Cuts Stories for Apple (A...,New York Times (NYT) Cuts Stories for Apple (A...,2019-03-18,https://news.google.com/rss/articles/CBMicmh0d...,"{'href': 'https://www.bloomberg.com', 'title':...",Are you a robot?,Why did this happen?\n\nPlease make sure your ...
1,"Apple lands the WSJ for its forthcoming ""Netfl...","Apple lands the WSJ for its forthcoming ""Netfl...",2019-03-22,https://news.google.com/rss/articles/CBMiWWh0d...,"{'href': 'https://www.businessinsider.com', 't...",Apple's news service just landed The Wall Stre...,This is an excerpt from a story delivered excl...
2,The Biggest Threats Facing Tech Stocks Now - I...,The Biggest Threats Facing Tech Stocks Now In...,2019-03-22,https://news.google.com/rss/articles/CBMiT2h0d...,"{'href': 'https://www.investopedia.com', 'titl...",The Biggest Threats Facing Tech Stocks Now,It would appear that the good old days are bac...
3,Apple (AAPL) Driverless Cars Need More Human I...,Apple (AAPL) Driverless Cars Need More Human I...,2019-02-13,https://news.google.com/rss/articles/CBMicGh0d...,"{'href': 'https://www.bloomberg.com', 'title':...",Are you a robot?,Why did this happen?\n\nPlease make sure your ...
4,Apple and Fitbit are leaders in the growing sm...,Apple and Fitbit are leaders in the growing sm...,2019-02-13,https://news.google.com/rss/articles/CBMiW2h0d...,"{'href': 'https://news.alphastreet.com', 'titl...",Apple and Fitbit are leaders in the growing sm...,The US smartwatch market has grown significant...


In [20]:
news.drop_duplicates(["url"])

,title,description,published date,url,publisher,fetched_title,fetched_body
0,New York Times (NYT) Cuts Stories for Apple (A...,New York Times (NYT) Cuts Stories for Apple (A...,2019-03-18,https://news.google.com/rss/articles/CBMicmh0d...,"{'href': 'https://www.bloomberg.com', 'title':...",Are you a robot?,Why did this happen?\n\nPlease make sure your ...
1,"Apple lands the WSJ for its forthcoming ""Netfl...","Apple lands the WSJ for its forthcoming ""Netfl...",2019-03-22,https://news.google.com/rss/articles/CBMiWWh0d...,"{'href': 'https://www.businessinsider.com', 't...",Apple's news service just landed The Wall Stre...,This is an excerpt from a story delivered excl...
2,The Biggest Threats Facing Tech Stocks Now - I...,The Biggest Threats Facing Tech Stocks Now In...,2019-03-22,https://news.google.com/rss/articles/CBMiT2h0d...,"{'href': 'https://www.investopedia.com', 'titl...",The Biggest Threats Facing Tech Stocks Now,It would appear that the good old days are bac...
3,Apple (AAPL) Driverless Cars Need More Human I...,Apple (AAPL) Driverless Cars Need More Human I...,2019-02-13,https://news.google.com/rss/articles/CBMicGh0d...,"{'href': 'https://www.bloomberg.com', 'title':...",Are you a robot?,Why did this happen?\n\nPlease make sure your ...
4,Apple and Fitbit are leaders in the growing sm...,Apple and Fitbit are leaders in the growing sm...,2019-02-13,https://news.google.com/rss/articles/CBMiW2h0d...,"{'href': 'https://news.alphastreet.com', 'titl...",Apple and Fitbit are leaders in the growing sm...,The US smartwatch market has grown significant...
...,...,...,...,...,...,...,...
952,Here's How Not Holding Apple (AAPL) Affected M...,Here's How Not Holding Apple (AAPL) Affected M...,"Tue, 13 Feb 2024 08:00:00 GMT",https://news.google.com/rss/articles/CBMiTWh0d...,"{'href': 'https://finance.yahoo.com', 'title':...",Here’s How Not Holding Apple (AAPL) Affected M...,"Mairs & Power, an investment advisor, released..."
953,AAPL Stock Weakness Sets Up Possible Bear Call...,AAPL Stock Weakness Sets Up Possible Bear Call...,"Tue, 13 Feb 2024 08:00:00 GMT",https://news.google.com/rss/articles/CBMiUGh0d...,"{'href': 'https://www.investors.com', 'title':...",AAPL Stock Weakness Sets Up Possible Bear Call...,Apple (AAPL) has been underperforming versus t...
954,Apple's (AAPL) Longest-Serving Designer Bart A...,Apple's (AAPL) Longest-Serving Designer Bart A...,"Tue, 13 Feb 2024 08:00:00 GMT",https://news.google.com/rss/articles/CBMidGh0d...,"{'href': 'https://www.bloomberg.com', 'title':...",Are you a robot?,Why did this happen?\n\nPlease make sure your ...
955,Apple (NASDAQ:AAPL) to Settle Chip Theft Lawsu...,Apple (NASDAQ:AAPL) to Settle Chip Theft Lawsu...,"Mon, 12 Feb 2024 08:00:00 GMT",https://news.google.com/rss/articles/CBMib2h0d...,"{'href': 'https://www.tipranks.com', 'title': ...",Apple (NASDAQ:AAPL) to Settle Chip Theft Lawsu...,Tech giant Apple (NASDAQ:AAPL) is planning to ...


In [21]:
import datetime
import dateutil.parser

In [22]:
news['published date'] = news['published date'].apply(lambda x: dateutil.parser.parse(x))


In [23]:
news['week'] = news['published date'].apply(lambda x: x.isocalendar()[1])
news['year'] = news['published date'].apply(lambda x:  x.isocalendar()[0])

# Sampling
We limit to 5 articles per week

In [24]:
news

,title,description,published date,url,publisher,fetched_title,fetched_body,week,year
0,New York Times (NYT) Cuts Stories for Apple (A...,New York Times (NYT) Cuts Stories for Apple (A...,2019-03-18 00:00:00,https://news.google.com/rss/articles/CBMicmh0d...,"{'href': 'https://www.bloomberg.com', 'title':...",Are you a robot?,Why did this happen?\n\nPlease make sure your ...,12,2019
1,"Apple lands the WSJ for its forthcoming ""Netfl...","Apple lands the WSJ for its forthcoming ""Netfl...",2019-03-22 00:00:00,https://news.google.com/rss/articles/CBMiWWh0d...,"{'href': 'https://www.businessinsider.com', 't...",Apple's news service just landed The Wall Stre...,This is an excerpt from a story delivered excl...,12,2019
2,The Biggest Threats Facing Tech Stocks Now - I...,The Biggest Threats Facing Tech Stocks Now In...,2019-03-22 00:00:00,https://news.google.com/rss/articles/CBMiT2h0d...,"{'href': 'https://www.investopedia.com', 'titl...",The Biggest Threats Facing Tech Stocks Now,It would appear that the good old days are bac...,12,2019
3,Apple (AAPL) Driverless Cars Need More Human I...,Apple (AAPL) Driverless Cars Need More Human I...,2019-02-13 00:00:00,https://news.google.com/rss/articles/CBMicGh0d...,"{'href': 'https://www.bloomberg.com', 'title':...",Are you a robot?,Why did this happen?\n\nPlease make sure your ...,7,2019
4,Apple and Fitbit are leaders in the growing sm...,Apple and Fitbit are leaders in the growing sm...,2019-02-13 00:00:00,https://news.google.com/rss/articles/CBMiW2h0d...,"{'href': 'https://news.alphastreet.com', 'titl...",Apple and Fitbit are leaders in the growing sm...,The US smartwatch market has grown significant...,7,2019
...,...,...,...,...,...,...,...,...,...
952,Here's How Not Holding Apple (AAPL) Affected M...,Here's How Not Holding Apple (AAPL) Affected M...,2024-02-13 08:00:00+00:00,https://news.google.com/rss/articles/CBMiTWh0d...,"{'href': 'https://finance.yahoo.com', 'title':...",Here’s How Not Holding Apple (AAPL) Affected M...,"Mairs & Power, an investment advisor, released...",7,2024
953,AAPL Stock Weakness Sets Up Possible Bear Call...,AAPL Stock Weakness Sets Up Possible Bear Call...,2024-02-13 08:00:00+00:00,https://news.google.com/rss/articles/CBMiUGh0d...,"{'href': 'https://www.investors.com', 'title':...",AAPL Stock Weakness Sets Up Possible Bear Call...,Apple (AAPL) has been underperforming versus t...,7,2024
954,Apple's (AAPL) Longest-Serving Designer Bart A...,Apple's (AAPL) Longest-Serving Designer Bart A...,2024-02-13 08:00:00+00:00,https://news.google.com/rss/articles/CBMidGh0d...,"{'href': 'https://www.bloomberg.com', 'title':...",Are you a robot?,Why did this happen?\n\nPlease make sure your ...,7,2024
955,Apple (NASDAQ:AAPL) to Settle Chip Theft Lawsu...,Apple (NASDAQ:AAPL) to Settle Chip Theft Lawsu...,2024-02-12 08:00:00+00:00,https://news.google.com/rss/articles/CBMib2h0d...,"{'href': 'https://www.tipranks.com', 'title': ...",Apple (NASDAQ:AAPL) to Settle Chip Theft Lawsu...,Tech giant Apple (NASDAQ:AAPL) is planning to ...,7,2024


In [25]:
sampled_news = news.groupby(["year","week"]).head(n=5)

In [26]:
sampled_news

,title,description,published date,url,publisher,fetched_title,fetched_body,week,year
0,New York Times (NYT) Cuts Stories for Apple (A...,New York Times (NYT) Cuts Stories for Apple (A...,2019-03-18 00:00:00,https://news.google.com/rss/articles/CBMicmh0d...,"{'href': 'https://www.bloomberg.com', 'title':...",Are you a robot?,Why did this happen?\n\nPlease make sure your ...,12,2019
1,"Apple lands the WSJ for its forthcoming ""Netfl...","Apple lands the WSJ for its forthcoming ""Netfl...",2019-03-22 00:00:00,https://news.google.com/rss/articles/CBMiWWh0d...,"{'href': 'https://www.businessinsider.com', 't...",Apple's news service just landed The Wall Stre...,This is an excerpt from a story delivered excl...,12,2019
2,The Biggest Threats Facing Tech Stocks Now - I...,The Biggest Threats Facing Tech Stocks Now In...,2019-03-22 00:00:00,https://news.google.com/rss/articles/CBMiT2h0d...,"{'href': 'https://www.investopedia.com', 'titl...",The Biggest Threats Facing Tech Stocks Now,It would appear that the good old days are bac...,12,2019
3,Apple (AAPL) Driverless Cars Need More Human I...,Apple (AAPL) Driverless Cars Need More Human I...,2019-02-13 00:00:00,https://news.google.com/rss/articles/CBMicGh0d...,"{'href': 'https://www.bloomberg.com', 'title':...",Are you a robot?,Why did this happen?\n\nPlease make sure your ...,7,2019
4,Apple and Fitbit are leaders in the growing sm...,Apple and Fitbit are leaders in the growing sm...,2019-02-13 00:00:00,https://news.google.com/rss/articles/CBMiW2h0d...,"{'href': 'https://news.alphastreet.com', 'titl...",Apple and Fitbit are leaders in the growing sm...,The US smartwatch market has grown significant...,7,2019
...,...,...,...,...,...,...,...,...,...
952,Here's How Not Holding Apple (AAPL) Affected M...,Here's How Not Holding Apple (AAPL) Affected M...,2024-02-13 08:00:00+00:00,https://news.google.com/rss/articles/CBMiTWh0d...,"{'href': 'https://finance.yahoo.com', 'title':...",Here’s How Not Holding Apple (AAPL) Affected M...,"Mairs & Power, an investment advisor, released...",7,2024
953,AAPL Stock Weakness Sets Up Possible Bear Call...,AAPL Stock Weakness Sets Up Possible Bear Call...,2024-02-13 08:00:00+00:00,https://news.google.com/rss/articles/CBMiUGh0d...,"{'href': 'https://www.investors.com', 'title':...",AAPL Stock Weakness Sets Up Possible Bear Call...,Apple (AAPL) has been underperforming versus t...,7,2024
954,Apple's (AAPL) Longest-Serving Designer Bart A...,Apple's (AAPL) Longest-Serving Designer Bart A...,2024-02-13 08:00:00+00:00,https://news.google.com/rss/articles/CBMidGh0d...,"{'href': 'https://www.bloomberg.com', 'title':...",Are you a robot?,Why did this happen?\n\nPlease make sure your ...,7,2024
955,Apple (NASDAQ:AAPL) to Settle Chip Theft Lawsu...,Apple (NASDAQ:AAPL) to Settle Chip Theft Lawsu...,2024-02-12 08:00:00+00:00,https://news.google.com/rss/articles/CBMib2h0d...,"{'href': 'https://www.tipranks.com', 'title': ...",Apple (NASDAQ:AAPL) to Settle Chip Theft Lawsu...,Tech giant Apple (NASDAQ:AAPL) is planning to ...,7,2024


In [27]:
sampled_news["fetched_body"] = sampled_news["fetched_body"].convert_dtypes(convert_string=True)
sampled_news = sampled_news.dropna(inplace=False)

/var/folders/b2/kj73pqdx4sxdntx6q_0yhbqh0000gp/T/ipykernel_67420/4193067446.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sampled_news["fetched_body"] = sampled_news["fetched_body"].convert_dtypes(convert_string=True)


In [28]:
sampled_news["fetched_body"].apply(lambda x: x.upper())

0      WHY DID THIS HAPPEN?\n\nPLEASE MAKE SURE YOUR ...
1      THIS IS AN EXCERPT FROM A STORY DELIVERED EXCL...
2      IT WOULD APPEAR THAT THE GOOD OLD DAYS ARE BAC...
3      WHY DID THIS HAPPEN?\n\nPLEASE MAKE SURE YOUR ...
4      THE US SMARTWATCH MARKET HAS GROWN SIGNIFICANT...
                             ...                        
952    MAIRS & POWER, AN INVESTMENT ADVISOR, RELEASED...
953    APPLE (AAPL) HAS BEEN UNDERPERFORMING VERSUS T...
954    WHY DID THIS HAPPEN?\n\nPLEASE MAKE SURE YOUR ...
955    TECH GIANT APPLE (NASDAQ:AAPL) IS PLANNING TO ...
956    MONDAY, FEBRUARY 12, 2024 11:06 AM | INVESTORS...
Name: fetched_body, Length: 893, dtype: object

In [29]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
import langchain

In [30]:
gpt = ChatOpenAI(model="gpt-3.5-turbo")

# Run Sample through GPT (Remote Large Language Model)

In [31]:
news_summary_prompt = ChatPromptTemplate.from_template("Summarize the following noisy news data and identify keywords of the news relevant to a Apple stock analyst. Provide a formatted answer as such Summary: ..., Keywords: ... News: {news}")
news_summary_chain = news_summary_prompt | gpt | StrOutputParser()
gpt_results = news_summary_chain.invoke({"news": sampled_news["fetched_body"].iloc[-1]})
print(gpt_results)

Summary: Analysts predict a 9.95% increase for Apple Inc (AAPL) over the next twelve months, and they give the stock an average rating of Strong Buy. The stock also has an Analyst Ranking of 48, signifying it ranks higher than 48% of stocks according to data from InvestorsObserver.

Keywords: Apple Inc, AAPL, increase, 9.95%, twelve months, average rating, Strong Buy, Analyst Ranking, 48, InvestorsObserver.


Using the delay of the response from the remote LLM as a natural way of rate limiting the system to avoid running into gpt rate cap

In [32]:
def batched(iter, batch_size):
    temp = []
    for i in iter:
        temp.append(i)
        if len(temp) == batch_size:
            yield temp
            temp = []
    yield temp

In [35]:
import time
res = []
for batch in batched(sampled_news["fetched_body"], 10):
    print("Starting Batch")
    morphed_batch = [{"news": article} for article in batch]
    res.extend(news_summary_chain.batch(morphed_batch))
    time.sleep(65)
res

Starting Batch
Starting Batch
Starting Batch
Starting Batch
Starting Batch
Starting Batch
Starting Batch
Starting Batch
Starting Batch
Starting Batch
Starting Batch
Starting Batch
Starting Batch
Starting Batch
Starting Batch
Starting Batch
Starting Batch
Starting Batch
Starting Batch
Starting Batch
Starting Batch
Starting Batch


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
sampled_news["gpt_results"] = res

In [ ]:
sampled_news.to_csv(".data/processed_news.csv")